In [1]:
from mlflow.tracking import MlflowClient

mlflow_tracking_uri = "sqlite:///../mlflow.db"
client = MlflowClient(tracking_uri=mlflow_tracking_uri)

In [5]:
client.create_experiment("model_registry_experiment")

'3'

In [2]:
from mlflow.entities import ViewType
runs = client.search_runs(
    experiment_ids=2,
    filter_string="",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metric.rmse ASC"]
    )
for run in runs:
    print(f"{run.info.run_id}, rmse: {run.data.metrics['rmse']:.15f}")

ed3365ce6c7a473fa7f6bb12cf294e48, rmse: 0.100694699545293
3139dc04d28249a9b469cb7775cf1cf8, rmse: 0.100694699545293
8f785651d3744786991b02daa305f2ed, rmse: 0.100694699545293
27504aedc0ed48aa9a5109550f08a027, rmse: 0.101063081544218
9e0ac1d2e0bc4e88b8a17ebfb95803a3, rmse: 0.101100944842802


In [3]:
client.get_latest_versions(name="nyctaxi-duration-model")

/tmp/ipykernel_1905/1358372835.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.get_latest_versions(name="nyctaxi-duration-model")


[<ModelVersion: aliases=[], creation_timestamp=1737152402292, current_stage='Production', description='', last_updated_timestamp=1737153548092, name='nyctaxi-duration-model', run_id='ed3365ce6c7a473fa7f6bb12cf294e48', run_link='', source='/workspaces/taxiML/02/mlruns/2/ed3365ce6c7a473fa7f6bb12cf294e48/artifacts/models_mlflow', status='READY', status_message=None, tags={}, user_id=None, version=1>,
 <ModelVersion: aliases=[], creation_timestamp=1737153097261, current_stage='Staging', description='', last_updated_timestamp=1737153345698, name='nyctaxi-duration-model', run_id='5b887b8ceafd4ce2a8ddd218b42be769', run_link='', source='models:/nyctaxi-duration-model/3', status='READY', status_message=None, tags={}, user_id=None, version=4>,
 <ModelVersion: aliases=[], creation_timestamp=1737152502830, current_stage='Archived', description='', last_updated_timestamp=1737153548092, name='nyctaxi-duration-model', run_id='5b887b8ceafd4ce2a8ddd218b42be769', run_link='', source='/workspaces/taxiML/

In [5]:
from datetime import datetime

client.transition_model_version_stage(
    name="nyctaxi-duration-model",
    version="2",
    stage="Production",
    archive_existing_versions=False,
)

client.update_model_version(
    name="nyctaxi-duration-model",
    version=2,
    description=f"model version 2 was transitioned to production on {datetime.today().date()}"
)

/tmp/ipykernel_1905/2183245769.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1737152440299, current_stage='Production', description='model version 2 was transitioned to production on 2025-01-18', last_updated_timestamp=1737231839818, name='nyctaxi-duration-model', run_id='df330795a1cd45db9b7d70e37fd7aa6b', run_link='', source='/workspaces/taxiML/02/mlruns/2/df330795a1cd45db9b7d70e37fd7aa6b/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [6]:
client.get_experiment(3)

<Experiment: artifact_location='/workspaces/taxiML/02/mlruns/3', creation_time=1737154076133, experiment_id='3', last_update_time=1737154076133, lifecycle_stage='active', name='model_registry_experiment', tags={}>

In [7]:
from sklearn.metrics import root_mean_squared_error
import pandas as pd
from utils import preprocess

test_data = pd.read_parquet("/workspaces/taxiML/data/green_tripdata_2021-03.parquet")
x, label = preprocess(test_data)

/workspaces/taxiML/02/utils.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['trip_type'].fillna(data.trip_type.mode().item(), inplace=True)
/workspaces/taxiML/02/utils.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inpl